#**Keras: 케라스**

케라스는 딥러닝 패키지를 편리하게 사용하기 위해 만들어진 래퍼(wrapper)이다.   
대표적인 딥러닝 패키지인 텐서플로, 씨아노 등을 사용해 신경망을 구현하려면 많은 양의 코드가 필요하다.   

In [ ]:
!pip install tensorflow_gpu==2.0.0

In [11]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

(train_data_all, train_target_all), (test_data, test_target) = tf.keras.datasets.fashion_mnist.load_data()
train_data, val_data, train_target, val_target = train_test_split(train_data_all, train_target_all, stratify=train_target_all, test_size=0.2, random_state=42)
train_data = train_data / 255
val_data = val_data / 255
train_data = train_data.reshape(-1, 784)
val_data = val_data.reshape(-1, 784)
train_target_encoded = tf.keras.utils.to_categorical(train_target)
val_target_encoded = tf.keras.utils.to_categorical(val_target)

#**Keras's Class**

케라스는 인공신경망을 '직관적으로' 구현할 수 있다.   
케라스에는 신경망 모델을 만드는 Sequential 클래스와 연결층을 만드는 Dense 클래스가 있다.

#**Sequential Class**

완전 연결(fully-connected) 신경망을 만들기 위해서는 Sequential과 Dense 클래스를 함께 사용한다.   
Sequential 객체를 생성할 때 층을 추가하는 방법과, 객체 생성 후 add() 메서드를 사용해 층을 추가하는 방법이 있다.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
model = Sequential([Dense(...), ...]) # Sequential 객체 생성 시 연결층 추가하는 방법

In [ ]:
dense = Dense(...)
model.add(dense)    # 객체 생성 뒤 add()로 층 추가하기

In [ ]:
model = Sequential()
model.add(Dense(...)) # add()로 추가할 때 Dense 클래스 만들기

#**Dense Class**

Dense 클래스에 전달해야 하는 첫 파라미터는 층의 **유닛(unit)**이다.   
은닉층의 유닛 개수를 100으로 지정하는 것으로 하자.

그 다음 전달해야 하는 파라미터는 활성화 함수 **activation**이다.   
activation을 따로 입력하지 않으면 기본값은 None으로 활성화 함수가 적용되지 않는다.   
activation에는 'sigmoid', 'softmax', 'tanh', 'relu' 등이 있다.

In [5]:
Dense(100, activation='sigmoid')

#**Optimizer & Loss**

모델을 훈련하기 위해서는 최적화 알고리즘이나 손실함수를 지정해야 한다.   
다중 분류에서의 최적화 알고리즘은 경사하강법, 손실함수는 크로스 엔트로피 손실함수를 사용한다.   


##**Optimizer: 최적화 알고리즘**
Sequential 클래스의 compile() 메서드를 사용해 최적화 알고리즘과 손실함수를 지정한다.   
optimizer 파라미터로 최적화 알고리즘을 지정한다.   
'sgd'를 입력하면 기본 경사 하강법을 최적화 알고리즘으로 사용한다.(이때 학습률의 기본값은 0.01이다.)

In [ ]:
model.compile(optimizer='sgd', ...)

##**Loss: 손실함수**

loss 파라미터를 사용해 손실함수를 지정한다.   
지금까지 공부한 손실함수는 제곱오차, 로지스틱, 크로스 엔트로피가 있다.   
'mse'는 제곱오차, 'binary_crossentropy'는 로지스틱, 'categorical_crossentropy'는 크로스 엔트로피이다.   
지금은 다중 분류 신경망을 구현할 것이므로 categorical_crossentropy로 지정한다.

In [ ]:
model.compile(optimizer='sgd', loss='categorical_crossentropy')

#**모델 훈련 및 예측**

Sequential 클래스의 fit() 메서드를 통해 훈련하고,   
predict() 메서드를 통해 예측한다.   
모델을 테스트 세트나 검증세트에서 평가할 때에는 evaluate() 메서드를 사용한다.

In [ ]:
model = Sequential()  # 객체 생성
model.add(Dense(...)) # 연결층 추가
model.add(Dense(...))
model.compile(optimizer='...', loss='...')  # 최적화 알고리즘 및 손실함수 지정
model.fit(x, y, epochs=...) # 훈련
model.predict(x)  # 예측
model.evaluate(x, y)  # 평가

#**Keras로 다중 분류 신경망을 만들어보자**
케라스를 이용해 다중 분류 신경망을 만든 다음 패션 MNIST 데이터셋을 이용해 훈련시켜보자.

1. Sequential()로 객체를 생성한 다음 Dense() 연결층을 추가해주자.   

2. 이때 앞에서 MultiClassNetwork 클래스에서는 100개의 은닉층과 10개의 출력층이 있었으므로, Dense에 각각 100, 10을 넣어주자.   
또한 은닉층의 활성화 함수는 시그모이드, 출력층의 활성화 함수는 소프트맥스 였으므로 각각 맞게 입력해주자.

3. 여기서 은닉층에는 input_shape 파라미터로 입력 데이터의 크기를 지정해야 한다.   
앞에서 28x28 사이즈의 이미지를 reshape 하여 (784,) 사이즈로 만들었기 때문에   
input_shape=(784, )를 입력해주자.

4. compile 메서드에 optimizer와 loss를 각각 sgd, categorical_crossentropy로 지정하자.
여기서 matrics 파라미터는 훈련 과정 기록으로 '정확도'를 남기기 위해 추가한다.   
matrics를 지정하지 않으면 History 객체에 기본값으로 손실값이 기록되지만   
'accuracy'를 넣어주어 정확도를 기록하도록 했다.




In [6]:
model = Sequential()  # 객체 생성하기
model.add(Dense(100, activation='sigmoid', input_shape=(784, ))) # 은닉층 추가
model.add(Dense(10, activation='softmax'))  # 출력층 추가
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

fit() 메서드로 40번의 훈련을 한다.   
이때 validation_data 파라미터로 검증세트를 튜플로 넘겨줄 수 있다.   
fit() 메서드는 훈련세트와 검증세트에서 측정한 값들을 History 클래스 객체에 담아 반환한다.

In [12]:
history = model.fit(train_data, train_target_encoded, epochs=40, validation_data=(val_data, val_target_encoded))

Train on 48000 samples, validate on 12000 samples
Epoch 1/40
48000/48000 [==============================] - 5s 106us/sample - loss: 1.3841 - accuracy: 0.6487 - val_loss: 0.9529 - val_accuracy: 0.7329
Epoch 2/40
48000/48000 [==============================] - 5s 97us/sample - loss: 0.8381 - accuracy: 0.7416 - val_loss: 0.7477 - val_accuracy: 0.7602
Epoch 3/40
48000/48000 [==============================] - 4s 93us/sample - loss: 0.7095 - accuracy: 0.7642 - val_loss: 0.6609 - val_accuracy: 0.7777
Epoch 4/40
48000/48000 [==============================] - 4s 91us/sample - loss: 0.6449 - accuracy: 0.7804 - val_loss: 0.6120 - val_accuracy: 0.7917
Epoch 5/40
48000/48000 [==============================] - 5s 94us/sample - loss: 0.6028 - accuracy: 0.7945 - val_loss: 0.5732 - val_accuracy: 0.8050
Epoch 6/40
48000/48000 [==============================] - 4s 87us/sample - loss: 0.5722 - accuracy: 0.8036 - val_loss: 0.5473 - val_accuracy: 0.8121
Epoch 7/40
48000/48000 [==============================]

In [13]:
print(history.history.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
